In [19]:
# 필요한 라이브러리 설치
# !pip install requests beautifulsoup4 pandas tqdm

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from tqdm.notebook import tqdm  # 주피터 노트북용 진행 표시줄

class SmitherySimpleCrawler:
    def __init__(self):
        """
        Initialize the Smithery.ai crawler
        """
        # 쿠키와 헤더 설정
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://smithery.ai/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0',
        }
        self.session = requests.Session()
        
        # 모든 서버 데이터를 저장할 리스트
        self.all_data = []
    
    def random_sleep(self, min_seconds=1, max_seconds=3):
        """
        Sleep for a random amount of time
        
        Args:
            min_seconds (float): Minimum sleep time in seconds
            max_seconds (float): Maximum sleep time in seconds
        """
        sleep_time = random.uniform(min_seconds, max_seconds)
        print(f"다음 요청까지 {sleep_time:.2f}초 대기 중...")
        time.sleep(sleep_time)
    
    def get_page(self, page_number):
        """
        Get the HTML content of a specific page
        
        Args:
            page_number (int): Page number to fetch
        
        Returns:
            BeautifulSoup: Parsed HTML content
        """
        url = f"https://smithery.ai/?q=is%3Adeployed&page={page_number}"
        
        try:
            print(f"\n페이지 {page_number} 로드 중: {url}")
            response = self.session.get(url, headers=self.headers)
            
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
            else:
                print(f"Error: HTTP {response.status_code} - {url}")
                return None
                
        except Exception as e:
            print(f"페이지 로드 중 오류 발생: {str(e)}")
            return None
    
    def extract_server_data(self, soup):
        """
        Extract server data from a BeautifulSoup object
        
        Args:
            soup (BeautifulSoup): Parsed HTML content
        
        Returns:
            list: List of dictionaries containing server data
        """
        servers_data = []
        
        if soup is None:
            return servers_data
        
        try:
            # 카드를 찾기 위한 다양한 선택자 시도
            server_cards = (
                soup.select('.card') or 
                soup.select('[class*="card"]') or 
                soup.select('a[href*="/smithery-ai/"]') or
                soup.select('a[href*="smithery.ai"]') or
                soup.select('main > div > div > div > div > a') or
                # 추가 선택자 - 가능한 모든 서버 카드 포함
                soup.select('div > a[href]')
            )
            
            print(f"발견된 서버 카드: {len(server_cards)}개")
            
            for card in server_cards:
                server = {}
                
                # 서버 이름 추출 시도
                name_element = (
                    card.select_one('h2') or 
                    card.select_one('h3') or 
                    card.select_one('h3 > span') or
                    card.select_one('div > h3') or
                    card.select_one('div > h3 > span') or
                    card.select_one('span[class*="title"]') or
                    card.select_one('div[class*="title"]')
                )
                
                if name_element:
                    server['name'] = name_element.get_text().strip()
                
                # 서버 핸들 추출 (@로 시작하는 텍스트)
                texts = card.get_text().split('\n')
                for text in texts:
                    text = text.strip()
                    if text.startswith('@'):
                        server['handle'] = text
                        break
                
                # 설명 추출
                desc_element = (
                    card.select_one('p') or 
                    card.select_one('div > p') or
                    card.select_one('div[class*="description"]')
                )
                
                if desc_element and not desc_element.get_text().strip().startswith('@'):
                    server['description'] = desc_element.get_text().strip()
                else:
                    # 설명을 찾을 수 없는 경우 모든 텍스트 블록을 검사
                    for p in card.select('p'):
                        text = p.get_text().strip()
                        if text and not text.startswith('@'):
                            server['description'] = text
                            break
                
                # URL 추출
                if card.name == 'a' and card.has_attr('href'):
                    url = card['href']
                    if not url.startswith('http'):
                        url = f"https://smithery.ai{url}"
                    server['url'] = url
                else:
                    url_element = card.select_one('a')
                    if url_element and url_element.has_attr('href'):
                        url = url_element['href']
                        if not url.startswith('http'):
                            url = f"https://smithery.ai{url}"
                        server['url'] = url
                
                # 유형 및 사용량 추출
                for span in card.select('span'):
                    text = span.get_text().strip()
                    if text in ['Remote', 'Local']:
                        server['type'] = text
                    elif text.endswith('k') and any(c.isdigit() for c in text):
                        server['usage_count'] = text
                
                # 이름이나 핸들이 있는 서버만 추가
                if server.get('name') or server.get('handle'):
                    # URL이 실제 서버와 관련된 것인지 확인 (필터링)
                    if server.get('url') and ('/smithery-ai/' in server.get('url') or '/server/' in server.get('url')):
                        servers_data.append(server)
                        print(f"  - 서버 데이터 추출: {server.get('name', server.get('handle', '알 수 없음'))}")
            
            return servers_data
            
        except Exception as e:
            print(f"서버 데이터 추출 중 오류 발생: {str(e)}")
            import traceback
            traceback.print_exc()
            return servers_data
    
    def extract_server_details(self, url):
        """
        Extract additional details from a server's page
        
        Args:
            url (str): URL of the server's page
        
        Returns:
            dict: Additional details about the server
        """
        details = {}
        
        try:
            self.random_sleep(0.5, 1.5)  # 짧은 대기 시간
            
            response = self.session.get(url, headers=self.headers)
            
            if response.status_code != 200:
                print(f"  - 서버 상세 정보 로드 실패: HTTP {response.status_code}")
                return details
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 라이선스 정보 추출
            license_elements = soup.find_all(string=re.compile('License', re.I))
            for element in license_elements:
                parent = element.parent
                if parent and parent.name in ['dt', 'h3', 'h4', 'strong', 'b', 'span']:
                    sibling = parent.find_next_sibling()
                    if sibling:
                        details['license'] = sibling.get_text().strip()
                        break
            
            # 출시일 추출
            publish_elements = soup.find_all(string=re.compile('Published', re.I))
            for element in publish_elements:
                parent = element.parent
                if parent and parent.name in ['dt', 'h3', 'h4', 'strong', 'b', 'span']:
                    sibling = parent.find_next_sibling()
                    if sibling:
                        details['published_date'] = sibling.get_text().strip()
                        break
            
            # 설치 명령어 추출
            install_elements = soup.find_all(string=re.compile('Installation|Install', re.I))
            for element in install_elements:
                parent = element.parent
                if parent:
                    code_block = parent.find_next('code') or parent.find_next('pre')
                    if code_block:
                        details['installation'] = code_block.get_text().strip()
                        break
            
            return details
            
        except Exception as e:
            print(f"  - 서버 상세 정보 추출 중 오류 발생: {str(e)}")
            return details
    
    def crawl_pages(self, start_page=1, end_page=45, get_details=False):
        """
        Crawl a range of pages from Smithery.ai
        
        Args:
            start_page (int): Page to start crawling from
            end_page (int): Page to stop crawling at
            get_details (bool): Whether to fetch additional details from each server's page
        
        Returns:
            list: All scraped data
        """
        # 결과를 초기화
        self.all_data = []
        
        # tqdm으로 진행 상황 표시
        for page_number in tqdm(range(start_page, end_page + 1), desc="페이지 크롤링"):
            # HTML 콘텐츠 가져오기
            soup = self.get_page(page_number)
            
            # 서버 데이터 추출
            page_data = self.extract_server_data(soup)
            
            # 상세 정보 추출 (선택 사항)
            if get_details and page_data:
                print("서버 상세 정보 추출 중...")
                for server in tqdm(page_data, desc="서버 상세 정보"):
                    if 'url' in server:
                        details = self.extract_server_details(server['url'])
                        server.update(details)
            
            # 결과 저장
            self.all_data.extend(page_data)
            
            print(f"페이지 {page_number} 완료: {len(page_data)}개 서버 추출 (누적: {len(self.all_data)}개)")
            
            # 다음 페이지 로드 전에 대기
            if page_number < end_page:
                self.random_sleep()
        
        return self.all_data
    
    def save_to_csv(self, filename="smithery_servers.csv"):
        """
        Save the scraped data to a CSV file
        
        Args:
            filename (str): Name of the CSV file to save
        
        Returns:
            pd.DataFrame: DataFrame containing the scraped data
        """
        df = pd.DataFrame(self.all_data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"데이터 저장 완료: {filename} ({len(self.all_data)}개 서버)")
        return df

# 실행 설정
START_PAGE = 1  # 시작 페이지
END_PAGE = 5    # 종료 페이지 (테스트용으로 낮게 설정)
GET_DETAILS = False  # 상세 정보 추출 여부
OUTPUT_FILE = "smithery_servers.csv"  # 결과 파일명

# 크롤러 실행 함수
def run_crawler(start_page=START_PAGE, end_page=END_PAGE, get_details=GET_DETAILS, output_file=OUTPUT_FILE):
    """
    Smithery.ai 크롤러를 실행하고 결과를 반환합니다.
    
    Args:
        start_page (int): 시작 페이지
        end_page (int): 종료 페이지
        get_details (bool): 상세 정보 추출 여부
        output_file (str): 결과 파일명
    
    Returns:
        pd.DataFrame: 크롤링 결과
    """
    # 크롤러 초기화
    crawler = SmitherySimpleCrawler()
    
    # 크롤링 수행
    crawler.crawl_pages(start_page=start_page, end_page=end_page, get_details=get_details)
    
    # 결과 저장 및 반환
    df = crawler.save_to_csv(output_file)
    
    return df

# 아래 주석을 제거하고 실행하세요
df = run_crawler(start_page=1, end_page=45)
df.head()  # 결과 확인

페이지 크롤링:   0%|          | 0/45 [00:00<?, ?it/s]


페이지 1 로드 중: https://smithery.ai/?q=is%3Adeployed&page=1
발견된 서버 카드: 42개
  - 서버 데이터 추출: Sequential Thinking
  - 서버 데이터 추출: Desktop Commander
  - 서버 데이터 추출: Github
  - 서버 데이터 추출: Think Tool Server
  - 서버 데이터 추출: Brave Search
  - 서버 데이터 추출: Magic MCP
  - 서버 데이터 추출: Browserbase
  - 서버 데이터 추출: Claude Code MCP Server
  - 서버 데이터 추출: VeyraX MCP
  - 서버 데이터 추출: Neon Database
  - 서버 데이터 추출: Exa Search
  - 서버 데이터 추출: Perplexity Search
  - 서버 데이터 추출: Fetch
  - 서버 데이터 추출: E2B
  - 서버 데이터 추출: Supabase MCP Server
  - 서버 데이터 추출: Figma API Integration
  - 서버 데이터 추출: Todoist MCP Server
  - 서버 데이터 추출: Gemini Thinking Server
  - 서버 데이터 추출: Notion API
  - 서버 데이터 추출: Stagehand
페이지 1 완료: 20개 서버 추출 (누적: 20개)
다음 요청까지 1.48초 대기 중...

페이지 2 로드 중: https://smithery.ai/?q=is%3Adeployed&page=2
발견된 서버 카드: 42개
  - 서버 데이터 추출: Linear MCP Server
  - 서버 데이터 추출: FFmpeg Video Processor
  - 서버 데이터 추출: Hyperbrowser
  - 서버 데이터 추출: Aindreyway Codex Keeper
  - 서버 데이터 추출: DuckDuckGo Search Server
  - 서버 데이터 추출: Playwright
  - 서버 데이터

,name,description,url,type,usage_count
0,Sequential Thinking,An MCP server implementation that provides a t...,https://smithery.ai/server/@smithery-ai/server...,Remote,681.30k
1,Desktop Commander,Execute terminal commands and manage files wit...,https://smithery.ai/server/@wonderwhy-er/deskt...,Local,331.87k
2,Github,"Access the GitHub API, enabling file operation...",https://smithery.ai/server/@smithery-ai/github,Remote,229.15k
3,Think Tool Server,Enhance your AI's reasoning capabilities with ...,https://smithery.ai/server/@PhillipRt/think-mc...,Remote,199.28k
4,Brave Search,Integrate web search and local search capabili...,https://smithery.ai/server/@smithery-ai/brave-...,Remote,155.66k


# web search

In [32]:
# 필요한 라이브러리 설치
# !pip install requests beautifulsoup4 pandas tqdm

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import urllib.parse
from tqdm.notebook import tqdm  # 주피터 노트북용 진행 표시줄

class SmitheryMultiQueryCrawler:
    def __init__(self):
        """
        Initialize the Smithery.ai crawler for multiple search queries
        """
        # 쿠키와 헤더 설정
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://smithery.ai/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0',
        }
        self.session = requests.Session()
        
        # 모든 서버 데이터를 저장할 리스트
        self.all_data = []
    
    def random_sleep(self, min_seconds=1, max_seconds=3):
        """
        Sleep for a random amount of time
        
        Args:
            min_seconds (float): Minimum sleep time in seconds
            max_seconds (float): Maximum sleep time in seconds
        """
        sleep_time = random.uniform(min_seconds, max_seconds)
        print(f"다음 요청까지 {sleep_time:.2f}초 대기 중...")
        time.sleep(sleep_time)
    
    def get_page(self, query, page_number=1):
        """
        Get the HTML content of a specific search query and page
        
        Args:
            query (str): Search query
            page_number (int): Page number to fetch
        
        Returns:
            BeautifulSoup: Parsed HTML content
        """
        # URL 인코딩 적용
        encoded_query = urllib.parse.quote(query)
        url = f"https://smithery.ai/?q={encoded_query}&page={page_number}"
        
        try:
            print(f"\n검색어 '{query}' - 페이지 {page_number} 로드 중: {url}")
            response = self.session.get(url, headers=self.headers)
            
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
            else:
                print(f"Error: HTTP {response.status_code} - {url}")
                return None
                
        except Exception as e:
            print(f"페이지 로드 중 오류 발생: {str(e)}")
            return None
    
    def get_max_pages(self, soup):
        """
        Get the maximum number of pages for the current search query
        
        Args:
            soup (BeautifulSoup): Parsed HTML content of the first page
        
        Returns:
            int: Maximum number of pages, defaults to 1 if not found
        """
        if soup is None:
            return 1
        
        try:
            # 페이지네이션 요소에서 마지막 페이지 번호 찾기
            pagination = soup.select('nav ul li a') or soup.select('nav ul a')
            max_page = 1
            
            for a in pagination:
                text = a.get_text().strip()
                if text.isdigit():
                    page_num = int(text)
                    max_page = max(max_page, page_num)
            
            # 페이지 번호가 발견되지 않으면 href 속성에서 찾기
            if max_page == 1:
                for a in pagination:
                    if a.has_attr('href'):
                        href = a['href']
                        match = re.search(r'page=(\d+)', href)
                        if match:
                            page_num = int(match.group(1))
                            max_page = max(max_page, page_num)
            
            print(f"검색 결과 최대 페이지 수: {max_page}")
            return max_page
        
        except Exception as e:
            print(f"최대 페이지 수 확인 중 오류 발생: {str(e)}")
            return 1
    
    def extract_server_data(self, soup, query):
        """
        Extract server data from a BeautifulSoup object
        
        Args:
            soup (BeautifulSoup): Parsed HTML content
            query (str): The search query used
        
        Returns:
            list: List of dictionaries containing server data
        """
        servers_data = []
        
        if soup is None:
            return servers_data
        
        try:
            # 카드를 찾기 위한 다양한 선택자 시도
            server_cards = (
                soup.select('.card') or 
                soup.select('[class*="card"]') or 
                soup.select('a[href*="/smithery-ai/"]') or
                soup.select('a[href*="smithery.ai"]') or
                soup.select('main > div > div > div > div > a') or
                # 추가 선택자 - 가능한 모든 서버 카드 포함
                soup.select('div > a[href]')
            )
            
            print(f"발견된 서버 카드: {len(server_cards)}개")
            
            for card in server_cards:
                server = {}
                
                # 검색 쿼리 추가
                server['search_query'] = query
                
                # 서버 이름 추출 시도
                name_element = (
                    card.select_one('h2') or 
                    card.select_one('h3') or 
                    card.select_one('h3 > span') or
                    card.select_one('div > h3') or
                    card.select_one('div > h3 > span') or
                    card.select_one('span[class*="title"]') or
                    card.select_one('div[class*="title"]')
                )
                
                if name_element:
                    server['name'] = name_element.get_text().strip()
                
                # 서버 핸들 추출 (@로 시작하는 텍스트)
                texts = card.get_text().split('\n')
                for text in texts:
                    text = text.strip()
                    if text.startswith('@'):
                        server['handle'] = text
                        break
                
                # 설명 추출
                desc_element = (
                    card.select_one('p') or 
                    card.select_one('div > p') or
                    card.select_one('div[class*="description"]')
                )
                
                if desc_element and not desc_element.get_text().strip().startswith('@'):
                    server['description'] = desc_element.get_text().strip()
                else:
                    # 설명을 찾을 수 없는 경우 모든 텍스트 블록을 검사
                    for p in card.select('p'):
                        text = p.get_text().strip()
                        if text and not text.startswith('@'):
                            server['description'] = text
                            break
                
                # URL 추출
                if card.name == 'a' and card.has_attr('href'):
                    url = card['href']
                    if not url.startswith('http'):
                        url = f"https://smithery.ai{url}"
                    server['url'] = url
                else:
                    url_element = card.select_one('a')
                    if url_element and url_element.has_attr('href'):
                        url = url_element['href']
                        if not url.startswith('http'):
                            url = f"https://smithery.ai{url}"
                        server['url'] = url
                
                # 유형 및 사용량 추출
                for span in card.select('span'):
                    text = span.get_text().strip()
                    if text in ['Remote', 'Local']:
                        server['type'] = text
                    elif text.endswith('k') and any(c.isdigit() for c in text):
                        server['usage_count'] = text
                
                # 이름이나 핸들이 있는 서버만 추가
                if server.get('name') or server.get('handle'):
                    servers_data.append(server)
                    print(f"  - 서버 데이터 추출: {server.get('name', server.get('handle', '알 수 없음'))}")
            
            return servers_data
            
        except Exception as e:
            print(f"서버 데이터 추출 중 오류 발생: {str(e)}")
            import traceback
            traceback.print_exc()
            return servers_data
    
    def extract_server_details(self, url):
        """
        Extract additional details from a server's page
        
        Args:
            url (str): URL of the server's page
        
        Returns:
            dict: Additional details about the server
        """
        details = {}
        
        try:
            self.random_sleep(0.5, 1.5)  # 짧은 대기 시간
            
            response = self.session.get(url, headers=self.headers)
            
            if response.status_code != 200:
                print(f"  - 서버 상세 정보 로드 실패: HTTP {response.status_code}")
                return details
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 라이선스 정보 추출
            license_elements = soup.find_all(string=re.compile('License', re.I))
            for element in license_elements:
                parent = element.parent
                if parent and parent.name in ['dt', 'h3', 'h4', 'strong', 'b', 'span']:
                    sibling = parent.find_next_sibling()
                    if sibling:
                        details['license'] = sibling.get_text().strip()
                        break
            
            # 출시일 추출
            publish_elements = soup.find_all(string=re.compile('Published', re.I))
            for element in publish_elements:
                parent = element.parent
                if parent and parent.name in ['dt', 'h3', 'h4', 'strong', 'b', 'span']:
                    sibling = parent.find_next_sibling()
                    if sibling:
                        details['published_date'] = sibling.get_text().strip()
                        break
            
            # 설치 명령어 추출
            install_elements = soup.find_all(string=re.compile('Installation|Install', re.I))
            for element in install_elements:
                parent = element.parent
                if parent:
                    code_block = parent.find_next('code') or parent.find_next('pre')
                    if code_block:
                        details['installation'] = code_block.get_text().strip()
                        break
            
            return details
            
        except Exception as e:
            print(f"  - 서버 상세 정보 추출 중 오류 발생: {str(e)}")
            return details
    
    def crawl_query(self, query, max_pages=None, get_details=False):
        """
        Crawl all pages for a specific search query
        
        Args:
            query (str): Search query to crawl
            max_pages (int, optional): Maximum number of pages to crawl. If None, automatically detect.
            get_details (bool): Whether to fetch additional details from each server's page
        
        Returns:
            list: Scraped data for this query
        """
        query_data = []
        
        # 첫 페이지 가져오기
        first_page_soup = self.get_page(query, 1)
        
        # 최대 페이지 수 확인
        if max_pages is None:
            max_pages = self.get_max_pages(first_page_soup)
        
        # 첫 페이지 데이터 추출
        first_page_data = self.extract_server_data(first_page_soup, query)
        query_data.extend(first_page_data)
        
        # 상세 정보 추출 (선택 사항)
        if get_details and first_page_data:
            print("서버 상세 정보 추출 중...")
            for server in tqdm(first_page_data, desc="서버 상세 정보"):
                if 'url' in server:
                    details = self.extract_server_details(server['url'])
                    server.update(details)
        
        print(f"페이지 1 완료: {len(first_page_data)}개 서버 추출")
        
        # 2페이지부터 크롤링
        for page_number in tqdm(range(2, max_pages + 1), desc=f"'{query}' 검색 결과 페이지 크롤링"):
            # 다음 페이지로 이동 전 대기
            self.random_sleep()
            
            # 페이지 로드
            soup = self.get_page(query, page_number)
            
            # 서버 데이터 추출
            page_data = self.extract_server_data(soup, query)
            
            # 상세 정보 추출 (선택 사항)
            if get_details and page_data:
                print("서버 상세 정보 추출 중...")
                for server in tqdm(page_data, desc="서버 상세 정보"):
                    if 'url' in server:
                        details = self.extract_server_details(server['url'])
                        server.update(details)
            
            # 결과 저장
            query_data.extend(page_data)
            
            print(f"페이지 {page_number} 완료: {len(page_data)}개 서버 추출 (쿼리 누적: {len(query_data)}개)")
        
        # 모든 데이터에 결과 추가
        self.all_data.extend(query_data)
        
        return query_data
    
    def crawl_multiple_queries(self, queries, max_pages=None, get_details=False):
        """
        Crawl multiple search queries
        
        Args:
            queries (list): List of search queries to crawl
            max_pages (int, optional): Maximum number of pages to crawl per query
            get_details (bool): Whether to fetch additional details from each server's page
        
        Returns:
            list: All scraped data
        """
        # 초기화
        self.all_data = []
        
        # 각 쿼리 크롤링
        for query in queries:
            print(f"\n\n=== 검색어 '{query}' 크롤링 시작 ===\n")
            self.crawl_query(query, max_pages, get_details)
            print(f"\n=== 검색어 '{query}' 크롤링 완료 (총 {len(self.all_data)}개 서버) ===")
        
        return self.all_data
    
    def save_to_csv(self, filename="smithery_all_queries.csv"):
        """
        Save the scraped data to a CSV file
        
        Args:
            filename (str): Name of the CSV file to save
        
        Returns:
            pd.DataFrame: DataFrame containing the scraped data
        """
        df = pd.DataFrame(self.all_data)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"데이터 저장 완료: {filename} ({len(self.all_data)}개 서버)")
        return df

# 설정값
QUERIES = [
    "is:deployed",  # 기본 검색어
    "web search"    # 추가 검색어
]
MAX_PAGES = 10       # 각 검색어당 최대 페이지 수 (None으로 설정하면 자동 감지)
GET_DETAILS = False  # 서버 상세 정보 추출 여부
OUTPUT_FILE = "smithery_all_servers.csv"  # 결과 파일명

# 크롤러 실행 함수
def run_crawler(queries=QUERIES, max_pages=MAX_PAGES, get_details=GET_DETAILS, output_file=OUTPUT_FILE):
    """
    Smithery.ai 크롤러를 실행하고 결과를 반환합니다.
    
    Args:
        queries (list): 크롤링할 검색어 목록
        max_pages (int): 각 검색어당 크롤링할 최대 페이지 수
        get_details (bool): 서버 상세 정보 추출 여부
        output_file (str): 결과 파일명
    
    Returns:
        pd.DataFrame: 크롤링 결과
    """
    # 크롤러 초기화
    crawler = SmitheryMultiQueryCrawler()
    
    # 크롤링 수행
    crawler.crawl_multiple_queries(queries, max_pages, get_details)
    
    # 결과 저장 및 반환
    df = crawler.save_to_csv(output_file)
    
    return df

# 아래 주석을 제거하고 실행하세요
# df = run_crawler(queries=["is:deployed", "web search"], max_pages=5)
# df.head()  # 결과 확인

# 개별 검색어 테스트
# df_web_search = run_crawler(queries=["web search"], max_pages=6, output_file="smithery_web_search.csv")
# df_web_search.head()

# df_advanced_web_browsing_automation = run_crawler(queries=["advanced web browsing automation"], max_pages=4, output_file="smithery_advanced_web_browsing_automation.csv")
# df_advanced_web_browsing_automation.head()

# df_dynamic_web_development = run_crawler(queries=["Tools and frameworks for building dynamic web applications and integrating real-time data"], max_pages=5, output_file="Tools and frameworks for building dynamic web applications and integrating real-time data.csv")
# df_dynamic_web_development.head() 


# df_ApplicationIntegrationToolst = run_crawler(queries=["Integrate applications with real-world data and external APIs for enhanced workflows and dynamic interactions. Explore servers enabling seamless integration with tools like Anki, Warpcast, and Beamlit for diverse operations."], max_pages=18, output_file="Application Integration Tools.csv")
# df_ApplicationIntegrationToolst.head() 

df_FinancialData_Analysis = run_crawler(queries=["Explore servers that provide advanced tools for tracking and analyzing cryptocurrency transactions, liquidity pools, and financial market data, including integrations with DeFi platforms and real-time financial metrics."], max_pages=5, output_file="Financial Data & Analysis.csv")
df_FinancialData_Analysis.head() 





=== 검색어 'Explore servers that provide advanced tools for tracking and analyzing cryptocurrency transactions, liquidity pools, and financial market data, including integrations with DeFi platforms and real-time financial metrics.' 크롤링 시작 ===


검색어 'Explore servers that provide advanced tools for tracking and analyzing cryptocurrency transactions, liquidity pools, and financial market data, including integrations with DeFi platforms and real-time financial metrics.' - 페이지 1 로드 중: https://smithery.ai/?q=Explore%20servers%20that%20provide%20advanced%20tools%20for%20tracking%20and%20analyzing%20cryptocurrency%20transactions%2C%20liquidity%20pools%2C%20and%20financial%20market%20data%2C%20including%20integrations%20with%20DeFi%20platforms%20and%20real-time%20financial%20metrics.&page=1
발견된 서버 카드: 42개
  - 서버 데이터 추출: Resources
  - 서버 데이터 추출: Crypto Price & Market Analysis Server
  - 서버 데이터 추출: Lunchmoney MCP Server
  - 서버 데이터 추출: MCP Server Starter
  - 서버 데이터 추출: Alpha Vantage Stock Server
 

'Explore servers that provide advanced tools for tracking and analyzing cryptocurrency transactions, liquidity…

다음 요청까지 2.35초 대기 중...

검색어 'Explore servers that provide advanced tools for tracking and analyzing cryptocurrency transactions, liquidity pools, and financial market data, including integrations with DeFi platforms and real-time financial metrics.' - 페이지 2 로드 중: https://smithery.ai/?q=Explore%20servers%20that%20provide%20advanced%20tools%20for%20tracking%20and%20analyzing%20cryptocurrency%20transactions%2C%20liquidity%20pools%2C%20and%20financial%20market%20data%2C%20including%20integrations%20with%20DeFi%20platforms%20and%20real-time%20financial%20metrics.&page=2
발견된 서버 카드: 42개
  - 서버 데이터 추출: Resources
  - 서버 데이터 추출: Alpha Vantage MCP Server
  - 서버 데이터 추출: Q-Anon Posts/Drops Server
  - 서버 데이터 추출: Jupiter Swap API Server
  - 서버 데이터 추출: Veri5ight
  - 서버 데이터 추출: PancakeSwap PoolSpy
  - 서버 데이터 추출: Crypto Price Query Server
  - 서버 데이터 추출: Hyperliquid MCP Server
  - 서버 데이터 추출: Crypto Market Maker Dashboard
  - 서버 데이터 추출: Whale Tracker
  - 서버 데이터 추출: DexScreener API Access
  - 서버 데이터 추출: Cry

,search_query,name,description,url,type,usage_count
0,Explore servers that provide advanced tools fo...,Resources,Building the operating system for AI,https://smithery.ai/,NaN,NaN
1,Explore servers that provide advanced tools fo...,Crypto Price & Market Analysis Server,Provide real-time cryptocurrency price data an...,https://smithery.ai/server/@truss44/mcp-crypto...,Remote,9.35k
2,Explore servers that provide advanced tools fo...,Lunchmoney MCP Server,Interact with your Lunchmoney transactions and...,https://smithery.ai/server/@leafeye/lunchmoney...,Remote,6.68k
3,Explore servers that provide advanced tools fo...,MCP Server Starter,Build a robust server to enable AI agents to i...,https://smithery.ai/server/@TheSethRose/mcp-se...,Remote,3.79k
4,Explore servers that provide advanced tools fo...,Alpha Vantage Stock Server,Access real-time and historical stock market d...,https://smithery.ai/server/@qubaomingg/stock-a...,Remote,NaN
